In [1]:
# Import necessary libraries
import xarray as xr
import rioxarray
import geopandas as gpd
import rasterio as rio
import matplotlib.pyplot as plt
import regionmask

In [2]:
# Load India shapefile
india_shapefile = gpd.read_file(r"C:\Users\djava\OneDrive\Documents\Oxford\Projects\india_rice_early_warning\4_data\RAW_DATA\SHAPEFILE\gadm36_IND_0.shp")

# Load India districts shapefile
india_districts_shapefile = gpd.read_file(r"C:\Users\djava\OneDrive\Documents\Oxford\Projects\india_rice_early_warning\4_data\RAW_DATA\SHAPEFILE\gadm36_IND_2.shp")

In [23]:
# query the shapefile to get raisen
india_districts_shapefile.query("NAME_2 == 'Raisen'")

,index,GID_0,NAME_0,GID_1,NAME_1,NL_NAME_1,GID_2,NAME_2,VARNAME_2,NL_NAME_2,TYPE_2,ENGTYPE_2,CC_2,HASC_2,geometry
321,321,IND,India,IND.19_1,Madhya Pradesh,NaN,IND.19.33_1,Raisen,NaN,NaN,District,District,NaN,IN.MP.RS,"POLYGON ((78.02436 22.80602, 78.02598 22.80997..."


In [4]:
india_districts_shapefile = india_districts_shapefile.reset_index()
india_districts_shapefile.head()

,index,GID_0,NAME_0,GID_1,NAME_1,NL_NAME_1,GID_2,NAME_2,VARNAME_2,NL_NAME_2,TYPE_2,ENGTYPE_2,CC_2,HASC_2,geometry
0,0,IND,India,IND.1_1,Andaman and Nicobar,NaN,IND.1.1_1,Nicobar Islands,NaN,NaN,District,District,NaN,IN.AN.NI,"MULTIPOLYGON (((93.78773 6.85264, 93.78849 6.8..."
1,1,IND,India,IND.1_1,Andaman and Nicobar,NaN,IND.1.2_1,North and Middle Andaman,NaN,NaN,District,District,NaN,IN.AN.NM,"MULTIPOLYGON (((92.93898 12.22386, 92.93916 12..."
2,2,IND,India,IND.1_1,Andaman and Nicobar,NaN,IND.1.3_1,South Andaman,NaN,NaN,District,District,NaN,IN.AN.SA,"MULTIPOLYGON (((92.47972 10.52056, 92.47945 10..."
3,3,IND,India,IND.2_1,Andhra Pradesh,NaN,IND.2.1_1,Anantapur,"Anantpur, Ananthapur",NaN,District,District,NaN,IN.AD.AN,"POLYGON ((77.71420 13.76079, 77.71314 13.75074..."
4,4,IND,India,IND.2_1,Andhra Pradesh,NaN,IND.2.2_1,Chittoor,Chitoor|Chittor,NaN,District,District,NaN,IN.AD.CH,"POLYGON ((78.46293 12.63537, 78.46190 12.63228..."


In [5]:
# Load Climate data (.nc format)
climate_data = xr.open_dataset(r"C:\Users\djava\OneDrive\Documents\Oxford\Projects\india_rice_early_warning\4_data\RAW_DATA\CDS\weather_data.nc")

In [6]:
# Convert temperature to celsius  
climate_data['t2m'] = climate_data['t2m'] - 273.15

In [27]:
climate_data

<xarray.Dataset>
Dimensions:    (longitude: 118, latitude: 111, expver: 2, time: 286)
Coordinates:
  * longitude  (longitude) float32 68.2 68.45 68.7 68.95 ... 96.95 97.2 97.45
  * latitude   (latitude) float32 35.5 35.25 35.0 34.75 ... 8.75 8.5 8.25 8.0
  * expver     (expver) int32 1 5
  * time       (time) datetime64[ns] 2000-01-01 2000-02-01 ... 2023-10-01
Data variables:
    t2m        (time, expver, latitude, longitude) float32 -6.052 -5.3 ... 27.31
    tp         (time, expver, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2023-11-19 00:13:23 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [7]:
# import regionmask

# # Create a mask for each district
# districts_mask = regionmask.mask_geopandas(india_districts_shapefile, climate_data.longitude, climate_data.latitude)


In [9]:
# climate_data

In [11]:
import regionmask
# Create mask of multiple regions from shapefile
india_mask = regionmask.mask_3D_geopandas(
        india_districts_shapefile,
        climate_data.longitude,
        climate_data.latitude,
        drop=True,
        numbers="index"
    )

In [12]:
# Apply the mask
climate_data_masked = climate_data.where(india_mask)

In [15]:
# Calculate mean values per district, per time step
mean_values = climate_data_masked.groupby('time').mean(dim=['latitude', 'longitude'])

In [16]:
df_mean_values = mean_values.to_dataframe().reset_index()

In [32]:
#filter df_mean_values where expver = 1 using query
# df_mean_values = 
df_mean_values.query('expver == 1').sample(50)

,time,expver,region,t2m,tp
346648,2022-04-01,1,83,31.518646,0.000084
333877,2021-06-01,1,296,33.325672,0.000586
246889,2015-11-01,1,273,27.560482,0.000481
288654,2018-07-01,1,513,26.514597,0.004343
362372,2023-04-01,1,234,26.577332,0.000585
12155,2000-10-01,1,488,30.684835,0.000020
333924,2021-06-01,1,343,27.563866,0.003930
155843,2010-01-01,1,84,14.511200,0.000083
3233,2000-03-01,1,653,26.371164,0.000059
375,2000-01-01,1,384,11.547777,0.000557


In [26]:
# query the df for raisen and 2022-01-01
df_mean_values.query("region == 321 & time == '2022-01-01'")

,time,expver,region,t2m,tp
342988,2022-01-01,1,321,16.143429,0.000532
343637,2022-01-01,5,321,NaN,NaN


In [12]:
climate_data

<xarray.Dataset>
Dimensions:    (longitude: 118, latitude: 111, expver: 2, time: 286)
Coordinates:
  * longitude  (longitude) float32 68.2 68.45 68.7 68.95 ... 96.95 97.2 97.45
  * latitude   (latitude) float32 35.5 35.25 35.0 34.75 ... 8.75 8.5 8.25 8.0
  * expver     (expver) int32 1 5
  * time       (time) datetime64[ns] 2000-01-01 2000-02-01 ... 2023-10-01
Data variables:
    t2m        (time, expver, latitude, longitude) float32 ...
    tp         (time, expver, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2023-11-19 00:13:23 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...